In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Домашнее задание №12

# 1 Задание №1

На любом языке программирования напишите программу, вычисляющую по заданной
таблице сопряжённости признаков

- статистику хи-квадрат,
- P-значение для критерия хи-квадрат Фишера-Пирсона,
- нормированные коэффициенты связи, основанные на статистике хи-квадрат,
- меры прогнозы Гутмана,
- коэффициенты контингенции и ассоциации (если таблица имеет размер 2 × 2).

## 1.1 Подготовка функций для решения задания №1

In [2]:
def get_p_value(stat_value, side='both', stat_type='norm', df=1):
    """ Расчет p-value для значения статистики
    
    Параметры
    ---------
    stat_value : float
      Значение статистики
    side : str
      Сторона теста для вычисления p-value
    stat_type : str
      Тип распределения значения статистики.
      Поддерживаемые значения: norm, ch2 и t
    df : int
      Количество степеней свободы. Только для chi2 и t
    Результат
    ---------
    p_val : float
    """
    
    side = str(side).lower().strip()
    assert side in ('both', 'left', 'right')
    stat_type = str(stat_type).lower().strip()
    assert stat_type in ('norm', 'chi2', 't')
    
    if stat_type == 'norm':
        stat_quantile = stats.norm.cdf(stat_value)
    elif stat_type == 'chi2':
        stat_quantile = stats.chi2.cdf(stat_value, df)
    else:
        stat_quantile = stats.t.cdf(stat_value, df)
    
    if side == 'both':
        p_val = np.min([2*stat_quantile, 2 - 2*stat_quantile])
    elif side == 'left':
        p_val = stat_quantile
    else:
        p_val = 1 - stat_quantile
        
    return p_val

In [3]:
def chi2_test(array, return_stats_norm=False):
    """Расчет статистики Хи-квадрат
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
    return_stats_norm : bool
      Если True, будут расчитаны дополнительные нормированные 
      коэффициенты статистики
      
    Результат
    ---------
    Если return_stats_norm == False:
    chi2, p_value
    
    иначе:
    chi2, p_value, chi2_norm
    
    chi2 : float
      Статистика
    p_value : float
      p-p_value для статистики
    chi2_norm : dict
      Словарь нормированных коэффициентов статистики:
      {
        # Коэффициент Пирсона
        'pearson': float,
        # Коэффициент Чупрова
        'chuprov': float,
        # Коэффициент Крамера
        'kramer': float,
        # Коэффициент среднеквадратичной сопряженности
        'sks': float,
    }
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
      
    
    if np.min(array) == 0:
        print('ВНИМАНИЕ: Найдены нулевые частоты в таблице контенгенций, тест может работать некорректно')
    elif np.min(array) <= 5:
        print('ВНИМАНИЕ: Найдены частоты менее 5 в таблице контенгенций, тест может работать некорректно')
        
    n = np.sum(array)
    # Сумма каждой строки отдельно
    n_i = np.sum(array, axis=1)
    # Сумма каждого столбца отдельно
    n_j = np.sum(array, axis=0)
    
    # Кол-во строк
    k = array.shape[0]
    # Кол-во столбцов
    m = array.shape[1]
    
    assert np.sum(n_i) == n
    assert np.sum(n_j) == n
    
    chi2 = []
    
    for i in range(array.shape[0]):
        temp = 0
        for j in range(array.shape[1]):
            a =  n_i[i] * n_j[j] / n
            temp += np.square(array[i,j] - a ) / a  
        chi2.append(temp)
        
    chi2 = np.sum(chi2)
    
    # Тест правосторонний
    p_value = get_p_value(chi2, side='right', stat_type='chi2', df=(k-1) * (m-1))
    
    if not return_stats_norm:
        return chi2, p_value
    
    chi2_norm = {
        'pearson': np.sqrt(chi2 / (n + chi2)),
        'chuprov': np.sqrt( chi2 / (n * np.sqrt( (k-1) * (m-1) )) ),
        'kramer': np.sqrt( chi2 / (n * np.min( [(k-1), (m-1)] )) ),
        'sks': np.sqrt( chi2 / n ),
    }
    
    assert all([0<=stat<=1 for stat in chi2_norm.values()]), 'Найдены нормированные статистики вне диапазона от 0 до 1'
    
    return chi2, p_value, chi2_norm

In [4]:
def gutman_test(array):
    """Мера прогноза Гутмана
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
      
    Результат
    ---------
    lambda : float
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
     
    lammdas = []
    
    n = np.sum(array)
    
    # Ищем вероятности ошибок прогнозов для обычной таблицы и транспонированной.
    # Результаты будем усреднять.
    for array_variant in (array, array.T):
    
        n_j = np.sum(array, axis=0)
        
        p_b1 = 1 - np.max(n_j) / n
        p_b2 = 1 - np.sum(np.max(array, axis=1) / n)
        
        lambda_variant = (p_b1 - p_b2) / p_b1
        lammdas.append(lambda_variant)
        
    return np.mean(lammdas)

In [5]:
def contingent_coef(array):
    """ Коэффициент контингенции
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
      
    Результат
    ---------
    contingent_coef : float
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
        
    assert array.shape[0] == 2 and array.shape[1] == 2
    
    a, b, c, d = array.flatten()
    
    return (a*d - b*c) / np.sqrt( (a+b) * (c+d) * (a+c) * (b+d) )

In [6]:
def association_coef(array):
    """ Коэффициент ассоциации
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
      
    Результат
    ---------
    contingent_coef : float
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
        
    assert array.shape[0] == 2 and array.shape[1] == 2
    
    a, b, c, d = array.flatten()
    
    return (a*b - b*c) / (a*d + b*c)


## 1.2 Решение задач в задании №1
### 1.2.1 Задача №1.1

По данным исследования «Мониторинг социальных и экономических перемен в России» составлена следующая таблица сопряженности:

In [7]:
df = pd.DataFrame({
    'Благоприятная': [12, 20, 11],
    'Напряженная': [48, 478, 160],
    'Взрывоопасная критическая': [47, 6666, 701]
}, index = ['Хорошее', 'Среднее', 'Плохое'])
df 

,Благоприятная,Напряженная,Взрывоопасная критическая
Хорошее,12,48,47
Среднее,20,478,6666
Плохое,11,160,701


**Вопрос А:** «Как бы Вы оценили в настоящее время материальное положение
вашей семьи?» (Хорошее, среднее, плохое).

**Вопрос В:** «Как бы Вы оценили в целом политическую обстановку в России?»
(Благополучная, напряжённая, взрывоопасная критическая).

Имеется ли зависимость между признаками А и В? Прокомментируйте характер
связи между А и В с помощью различных коэффициентов.

### Решение

Воспользуемся функцией chi2_test для получения всех необходимых данных:

In [8]:
chi2, p_value, chi2_norm = chi2_test(df, return_stats_norm=True)

In [9]:
alpha = 0.05

print("====================================")
print('chi2:', chi2)
print('p_value:', p_value)
for chi2_norm_coef_name, chi2_norm_coef_val in chi2_norm.items():
    print(f"{chi2_norm_coef_name}: {chi2_norm_coef_val}")
print('guttman:', gutman_test(df))

print("====================================")
if p_value < alpha:
    print(f"Между признаками имеется зависимость на уровне значимости {alpha}")
else:
    print(f"Между признаками отсутствует зависимость на уровне значимости {alpha}")

chi2: 585.4980121554853
p_value: 0.0
pearson: 0.258995938581778
chuprov: 0.18960749353865317
kramer: 0.18960749353865317
sks: 0.2681454888899323
guttman: 0.0013717421124842277
Между признаками имеется зависимость на уровне значимости 0.05


Тест определил наличие зависимости. P-value равно 0, поэтому с точки зрения теста вероятность зависимости очень высокая даже при меньшем пороге уровне значимости. При этом нормированные коэффициенты говорят о наличии очень слабой зависимости, так как они не достигают даже уровня 0.5

### 1.2.2 Задача №1.2

Изучается зависимость выбираемой абитуриентами специальности от пола. Было опрошено 480 человек и получены следующие результаты:

In [10]:
df = pd.DataFrame({
    'М': [168, 85],
    'Ж': [85, 135],
}, index = ['Естественные науки', 'Гуманитарные науки'])
df 

,М,Ж
Естественные науки,168,85
Гуманитарные науки,85,135


Можно ли на уровне доверия 0,95 считать, что между полом и выбором специальности есть связь? Оцените силу связи с помощью различных коэффициентов.

In [11]:
chi2, p_value, chi2_norm = chi2_test(df, return_stats_norm=True)

In [12]:
alpha = 0.05

print("====================================")
print('chi2:', chi2)
print('p_value:', p_value)
for chi2_norm_coef_name, chi2_norm_coef_val in chi2_norm.items():
    print(f"{chi2_norm_coef_name}: {chi2_norm_coef_val}")
print('guttman:', gutman_test(df))
print('contingent_coef:', contingent_coef(df))
print('association_coef:', association_coef(df))


print("====================================")
if p_value < alpha:
    print(f"Между признаками имеется зависимость на уровне значимости {alpha}")
else:
    print(f"Между признаками отсутствует зависимость на уровне значимости {alpha}")

chi2: 36.468067967004636
p_value: 1.5518788476498457e-09
pearson: 0.26754564947027903
chuprov: 0.2776679841897233
kramer: 0.2776679841897233
sks: 0.2776679841897233
guttman: 0.22727272727272743
contingent_coef: 0.2776679841897233
association_coef: 0.23591372680153821
Между признаками имеется зависимость на уровне значимости 0.05


Аналогично тест определил наличие зависимости. И точно также нормированные коэффициенты говорят  о наличии очень слабой зависимости, так как они не достигают даже уровня 0.5

# 2 Задание №2

На любом языке программирования напишите программу, которая по результатам наблюдений за двумя количественными признаками:
- вычисляет выборочный коэффициент корреляции и P-значение для критерия проверки гипотезы о нулевом значении коэффициента корреляции;
- вычисляет коэффициент Спирмена и P-значение для критерия Спирмена;
- вычисляет коэффициент Крамера и P-значение для критерия Крамера;
- разбивая диапазон количественных признаков на несколько интервалов строит по наблюдениям таблицу сопряжённости, вычисляет нормированные коэффициенты связи (Пирсона, Крамера и др.), меры прогноза Гутмана и P-значение для критерия хи-квадрат Фишера Пирсона.

## 2.1 Подготовка функций для решения задания №2

In [13]:
def pearson_test(array):
    """ Тест Пирсона (стандартный корреляционный)
    
    Можно применять только для данных с двумя наборами признаков.
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
      
    Результат
    ---------
    stat, p_value, r_xy : tuple
    
    stat : float
      Значение статистики
    p_value : float
      p-value для статистики
    r_xy : float
      Коэффициент корреляции
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
        
    assert array.shape[1] == 2
    
    n = array.shape[0] 
    
    means = array.mean(axis=0)
    array_centered = array - means
    
    r_xy = np.sum(array_centered[:, 0] * array_centered[:, 1]) / np.sqrt( np.sum(array_centered[:, 0]**2) 
                                                                          * np.sum(array_centered[:, 1]**2) )
    assert -1 <= r_xy <= 1
    
    stat = r_xy * np.sqrt(n - 2) / np.sqrt(1 - r_xy**2)
    p_value = get_p_value(stat, side='both', stat_type='t', df=n-2)
    
    return stat, p_value, r_xy

In [14]:
def spearman_test(array):
    """ Тест Спирмана
    
    Можно применять только для данных с двумя наборами признаков.
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
      
    Результат
    ---------
    stat, p_value, p_s : tuple
    
    stat : float
      Значение статистики
    p_value : float
      p-value для статистики
    p_s : float
      Коэффициент корреляции Спирмана
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
        
    assert array.shape[1] == 2
    
    n = array.shape[0]
    
    # Ранжируем каждый признак отдельно без сортировки
    rank_data_x = stats.rankdata(array[:, 0])
    rank_data_y = stats.rankdata(array[:, 1])
    
    # Находим сумму квадратов разниц рангов
    s = np.sum( ( rank_data_x - rank_data_y)**2 )
    
    # Коэффициент корреляции Спирмана
    p_s = 1 - 6*s / (n**3 - n)
    
    assert -1 <= p_s <= 1
    
    stat = np.sqrt(n - 1) * p_s
    p_value = get_p_value(stat, side='both', stat_type='norm')
    
    return stat, p_value, p_s

In [15]:
def kendall_test(array):
    """ Тест Кендалла
    
    Можно применять только для данных с двумя наборами признаков.
    
    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
      
    Результат
    ---------
    stat, p_value, p_s : tuple
    
    stat : float
      Значение статистики
    p_value : float
      p-value для статистики
    t : float
      Коэффициент корреляции Кендалла
    """
    
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
        
    assert array.shape[1] == 2
    
    n = array.shape[0]
    
    # Ранжируем каждый признак отдельно без сортировки
    rank_data_x = stats.rankdata(array[:, 0])
    rank_data_y = stats.rankdata(array[:, 1])
    
    # Получаем список пар рангов признаков [(rank_x1, rank_y1), (rank_x2, rank_y2), ...]
    rank_pairs = list(zip(rank_data_x, rank_data_y))
    
    # Сортируем список пар по признаку x от меньшего к большему
    sorted_rank_data = list(sorted(rank_pairs))
    
    # В итоге получаем отсортированные ранги признака Y по признаку X
    sorted_rank_data = np.array(list(zip(*sorted_rank_data)))
    # Для нашей формулы требуются только ранги признака Y
    sorted_rank_data_y = sorted_rank_data[1]
    
    # Кол-во элементов, у которых ранг выше текущей позиции
    y_up = 0
    # Кол-во элементов, у которых ранг ниже текущей позиции
    y_down = 0
    
    for idx in range(n-1):
        y_up += len(np.where(sorted_rank_data_y[idx+1:] > sorted_rank_data_y[idx])[0])
        y_down += len(np.where(sorted_rank_data_y[idx+1:] < sorted_rank_data_y[idx])[0])  
        
    s = y_up - y_down
    t = 2 * s / (n * (n - 1))
    
    assert -1 <= t <= 1
    
    stat = 3 * np.sqrt(n) * t / 2
    p_value = get_p_value(stat, side='both', stat_type='norm')
    
    return stat, p_value, t

In [16]:
def make_contingency_table(array, bins=2):
    """ Формирование таблицы сопряженности
    
    Функция разбивает на интервалы каждый признак по отдельности.
    Поддерживаются только массивы с двумя числовыми признаками.
    Формируется матрица, у которой количество строк - количество бинов первого признака,
    а количество столбцов - количество бинов второго признака.
    Берется значение с двумя признаками и счётчик увеличивается в той ячейке,
    в диапазон которых попадают оба значения.

    Параметры
    ---------
    array : np.ndarray, pd.DataFrame
      Массив данных
    bins : int
      Количество интервалов, на которые нужно разбить данные
      
    Результат
    ---------
    contingency_table : np.ndarray
      Имеет размер B x C,
      где С - кол-во столбцов в array и B - количество бинов
    """
    
    if isinstance(array, pd.DataFrame):
        array = array.to_numpy()
        
    if not isinstance(array, np.ndarray):
        array = np.array(array)
        
    assert array.shape[1] == 2
    
    # Здесь получаем интервалы для каждого отдельного признака: [интервалы_признак_1, интервалы_признак_2, .. ]
    categories_ranges = [pd.cut(features, bins=bins).categories for features in array.T]
    
    # Делаем таблицу сопряженности с нулевыми значениями размером bins x bins
    contingency_table = np.zeros(shape=[bins, bins])
    
    # Пробегаемся по каждой колонке и строке
    for row_idx in range(array.shape[0]):
        # Значение исходной таблицы
        value1 = array[row_idx, 0]
        value2 = array[row_idx, 1]
        # Находим к какому номеру диапазона относится значение
        category_1 = np.argmax([value1 in value_range for value_range in categories_ranges[0]])
        # Находим к какому номеру диапазона относится значение
        category_2 = np.argmax([value2 in value_range for value_range in categories_ranges[1]])
        # Увеличиваем значение на 1 для нужного номера диапазона
        contingency_table[category_1, category_2] += 1
        
    # Удаляем все строки, у которых только нулевые значения
    contingency_table = contingency_table[contingency_table.any(axis=1)]
    # Удаляем все столбцы, у которых только нулевые значения
    contingency_table = contingency_table[:, contingency_table.any(axis=0)]
             
    return contingency_table

## 2.2 Решение задач в задании №2
### 2.2.1 Задача №2.1

Группа пловцов из 15 человек принимает участие в местной спортивной олимпиаде. В программе состязаний два заплыва: 50 м вольным стилем и 100 м баттерфляем. 

Результаты соревнований следующие:

In [17]:
a = [22.49, 22.56, 23.45, 22.58, 24.3, 24.2, 23.47, 23.5, 24.48, 25.02, 23.04, 23.24, 25.2, 24.61, 26.02]
b = [52.93, 53.4, 53.7, 53.36, 61.8, 55.2, 53.54, 58.33, 60.4, 60.3, 54.28, 53.6, 62.24, 54.45, 61.52]

df = pd.DataFrame([a, b], columns=np.arange(1, len(a)+1), index=['50 м вольн.', '100 м батт.']).transpose()
df

,50 м вольн.,100 м батт.
1,22.49,52.93
2,22.56,53.40
3,23.45,53.70
4,22.58,53.36
5,24.30,61.80
6,24.20,55.20
7,23.47,53.54
8,23.50,58.33
9,24.48,60.40
10,25.02,60.30


Определите степень связи между результатами в разных заплывах.

In [18]:
alpha = 0.05
bins = 2

print("====================================\n")
for test_name, test_func in zip(['pearson', 'spearman', 'kendall'], [pearson_test, spearman_test, kendall_test]):
    stat, p_value, corr = test_func(df)
    print(f'{test_name}_stat:', stat)
    print(f'{test_name}_p_value:', p_value)
    print(f'{test_name}_corr:', corr, '\n')

contingency_table = make_contingency_table(df, bins=bins)

print("====================================\n")
print(f'Таблица сопряженности для {bins} бинов:\n')
print(contingency_table, '\n')

chi2, p_value, chi2_norm = chi2_test(contingency_table, return_stats_norm=True)

print('chi2:', chi2)
print('p_value:', p_value)
for chi2_norm_coef_name, chi2_norm_coef_val in chi2_norm.items():
    print(f"{chi2_norm_coef_name}: {chi2_norm_coef_val}")
print('guttman:', gutman_test(df))

if p_value < alpha:
    print(f"\nМежду признаками в таблице сопряженности имеется зависимость на уровне значимости {alpha}")
else:
    print(f"\nМежду признаками в таблице сопряженности отсутствует зависимость на уровне значимости {alpha}")


pearson_stat: 4.9431604784651
pearson_p_value: 0.0002686773575546475
pearson_corr: 0.8079169977411043 

spearman_stat: 3.300676337618441
spearman_p_value: 0.0009645208069288813
spearman_corr: 0.8821428571428571 

kendall_stat: 4.038968346759163
kendall_p_value: 5.3686808705766254e-05
kendall_corr: 0.6952380952380952 


Таблица сопряженности для 2 бинов:

[[8. 1.]
 [1. 5.]] 

ВНИМАНИЕ: Найдены частоты менее 5 в таблице контенгенций, тест может работать некорректно
chi2: 7.824074074074074
p_value: 0.005155485148346517
pearson: 0.5854905538443584
chuprov: 0.7222222222222222
kramer: 0.7222222222222222
sks: 0.7222222222222222
guttman: 7.484210060072233e-16

Между признаками в таблице сопряженности имеется зависимость на уровне значимости 0.05


### 2.2.2 Задача №2.2

В таблице приведены данные об урожайности пшеницы и картофеля на соседних полях. Что можно сказать о зависимости показателей урожайности картофеля и пшеницы?

In [19]:
a = [20.1, 23.6, 26.3, 19.9, 16.7, 23.2, 31.4, 33.5, 28.2, 35.3, 29.3]
b = [7.2, 7.1, 7.4, 6.1, 6.0, 7.3, 9.4, 9.2, 8.8, 10.4, 8.0]

df = pd.DataFrame([a, b], columns=np.arange(1926, 1937), index=['Пшеница', 'Картофель']).transpose()
df

,Пшеница,Картофель
1926,20.1,7.2
1927,23.6,7.1
1928,26.3,7.4
1929,19.9,6.1
1930,16.7,6.0
1931,23.2,7.3
1932,31.4,9.4
1933,33.5,9.2
1934,28.2,8.8
1935,35.3,10.4


In [20]:
alpha = 0.05
bins = 2

print("====================================\n")
for test_name, test_func in zip(['pearson', 'spearman', 'kendall'], [pearson_test, spearman_test, kendall_test]):
    stat, p_value, corr = test_func(df)
    print(f'{test_name}_stat:', stat)
    print(f'{test_name}_p_value:', p_value)
    print(f'{test_name}_corr:', corr, '\n')

contingency_table = make_contingency_table(df, bins=bins)

print("====================================\n")
print(f'Таблица сопряженности для {bins} бинов:\n')
print(contingency_table, '\n')

chi2, p_value, chi2_norm = chi2_test(contingency_table, return_stats_norm=True)

print('chi2:', chi2)
print('p_value:', p_value)
for chi2_norm_coef_name, chi2_norm_coef_val in chi2_norm.items():
    print(f"{chi2_norm_coef_name}: {chi2_norm_coef_val}")
print('guttman:', gutman_test(df))

if p_value < alpha:
    print(f"\nМежду признаками в таблице сопряженности имеется зависимость на уровне значимости {alpha}")
else:
    print(f"\nМежду признаками в таблице сопряженности отсутствует зависимость на уровне значимости {alpha}")


pearson_stat: 8.99104946703741
pearson_p_value: 8.607825483286646e-06
pearson_corr: 0.9485888243336862 

spearman_stat: 3.0185377665243625
spearman_p_value: 0.002539977411205818
spearman_corr: 0.9545454545454546 

kendall_stat: 4.251309958546467
kendall_p_value: 2.1252380079328503e-05
kendall_corr: 0.8545454545454545 


Таблица сопряженности для 2 бинов:

[[5. 0.]
 [2. 4.]] 

ВНИМАНИЕ: Найдены нулевые частоты в таблице контенгенций, тест может работать некорректно
chi2: 5.238095238095239
p_value: 0.02209745528422724
pearson: 0.5679618342470648
chuprov: 0.6900655593423543
kramer: 0.6900655593423543
sks: 0.6900655593423543
guttman: 0.0

Между признаками в таблице сопряженности имеется зависимость на уровне значимости 0.05
